<a href="https://colab.research.google.com/github/Anilesh05/Big_Data_Laboratory/blob/main/Find_grade_Using_Mapreduce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Downloading and installing hadoop***

In [1]:
!apt-get install openjdk-8-jdk
!wget https://dlcdn.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
!tar fx hadoop-3.3.6.tar.gz
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["HADOOP_HOME"] = "/content/hadoop-3.3.6"
!ln -s /content/hadoop-3.3.6/bin/* /usr/bin

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java libatk-wrapper-java-jni libfontenc1
  libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin libgtk2.0-common libice-dev librsvg2-common
  libsm-dev libxkbfile1 libxt-dev libxtst6 libxxf86dga1 openjdk-8-jdk-headless openjdk-8-jre
  openjdk-8-jre-headless x11-utils
Suggested packages:
  gvfs libice-doc libsm-doc libxt-doc openjdk-8-demo openjdk-8-source visualvm libnss-mdns
  fonts-nanum fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei
  fonts-indic mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java libatk-wrapper-java-jni libfontenc1
  libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin libgtk2.0-common libice-dev librsvg2-common
  libsm-dev libxkbfile1 libxt-dev libxtst6 libxxf86dga1 openjdk-

# ***Create mapper.py***

In [2]:
%%writefile mapper.py
#!/usr/bin/env python

import sys

# Skip the header row
header_skipped = False

# Input comes from STDIN (standard input)
for line in sys.stdin:
    # Remove leading and trailing whitespace
    line = line.strip()

    if not header_skipped:
        # Skip the header row
        header_skipped = True
        continue

    # Split the line into columns
    columns = line.split(',')
    if len(columns) == 3:
        student_id, name, marks = columns
        total_marks = sum(map(float, [marks]))
        # Determine the grade
        if total_marks >= 90:
            grade = "A+"
        elif total_marks >= 80:
            grade = "A"
        elif total_marks >= 70:
            grade = "B+"
        elif total_marks >= 60:
            grade = "B"
        elif total_marks >= 50:
            grade = "C"
        else:
            grade = "U"
        # Output the result
        print(f"{student_id},{name},{total_marks},{grade}")


Writing mapper.py


# ***Create Reducer.py***

In [39]:
%%writefile reducer.py
#!/usr/bin/env python

import sys

# Define column names
column_names = ["S-ID", "Name", "Total Marks", "Grade"]

# Print column headers
print('{:10} {:<11} {:<16} {:<15}'.format(*column_names))
print()

# Input comes from STDIN (standard input)
for line in sys.stdin:
    # Remove leading and trailing whitespace
    line = line.strip()
    # Split the line into columns
    student_id, name, total_marks, grade = line.split(',')
    # Output in table format
    print(f"{student_id:<10} {name:<15} {total_marks:<15} {grade}")


Overwriting reducer.py


# ***Create Input Directory***

In [4]:
!hdfs dfs -mkdir input

# ***Write input file***

In [5]:
%%writefile input/marksheet.txt

01,John,85
02,Emma,92
03,Michael,78
04,Sophia,60
05,William,88
06,Isabella,75
07,James,90
08,Olivia,82
09,Alexander,70
10,Ava,95
11,Ethan,72
12,Mia,86
13,Daniel,81
14,Charlotte,79
15,Joseph,94
16,Abigail,87
17,David,76
18,Emily,93
19,Benjamin,83
20,Harper,77
21,Matthew,91
22,Elizabeth,84
23,Jackson,34
24,Ella,89
25,Samuel,80
26,Avery,96
27,Henry,73
28,Scarlett,97
29,Gabriel,68
30,Victoria,98
31,Andrew,40
32,Grace,99
33,Ryan,65
34,Chloe,100
35,Lucas,66
36,Sofia,98
37,Christopher,67
38,Liam,97
39,Evelyn,64
40,Dylan,96

Writing input/marksheet.txt


# ***Run hadoop mapreduce***

In [41]:
!hadoop jar /content/hadoop-3.3.6/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar \
    -files mapper.py,reducer.py \
    -mapper mapper.py \
    -reducer reducer.py \
    -input input/marksheet.txt \
    -output output

2024-04-03 15:18:43,668 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2024-04-03 15:18:43,776 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2024-04-03 15:18:43,776 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2024-04-03 15:18:43,801 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2024-04-03 15:18:44,149 INFO mapred.FileInputFormat: Total input files to process : 1
2024-04-03 15:18:44,171 INFO mapreduce.JobSubmitter: number of splits:1
2024-04-03 15:18:44,384 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local382903436_0001
2024-04-03 15:18:44,384 INFO mapreduce.JobSubmitter: Executing with tokens: []
2024-04-03 15:18:44,965 INFO mapred.LocalDistributedCacheManager: Localized file:/content/mapper.py as file:/tmp/hadoop-root/mapred/local/job_local382903436_0001_0e9df431-fba9-4746-88a2-21905bdd2a2b/mapper.py
2024-04-03 15:18:45,035 INFO mapred.LocalDistributedCacheMana

# ***Display Output***

In [42]:
!cat output/part-00000

S-ID       Name        Total Marks      Grade          	
	
01         John            85.0            A	
02         Emma            92.0            A+	
03         Michael         78.0            B+	
04         Sophia          60.0            B	
05         William         88.0            A	
06         Isabella        75.0            B+	
07         James           90.0            A+	
08         Olivia          82.0            A	
09         Alexander       70.0            B+	
10         Ava             95.0            A+	
11         Ethan           72.0            B+	
12         Mia             86.0            A	
13         Daniel          81.0            A	
14         Charlotte       79.0            B+	
15         Joseph          94.0            A+	
16         Abigail         87.0            A	
17         David           76.0            B+	
18         Emily           93.0            A+	
19         Benjamin        83.0            A	
20         Harper          77.0            B+	
21       